In [256]:
import numpy as np
import pandas as pd

In [125]:
sequences = pd.read_csv('data/kit_sequenzen.csv', header=0)

In [126]:
annotation = pd.read_csv('data/kit_annotation.csv', header=0)

In [267]:
df = pd.DataFrame(columns=['ID', 'sequence', 'coordinates', 'start_coordinates', 'set_0',
                          'set_1', 'set_2', 'set_3', 'set_4', 'set_5'])

In [268]:
data = pd.read_csv('data/kit_annotation_clustered.csv', header=0)

In [269]:
all_items = []

for col in data.columns:
    if 'geschirr' not in col and 'head' not in col and 'x' in col:
        all_items.append(col[:-2])

In [270]:
ids = set(data['ID'])

In [271]:
df['ID'] = sorted(list(ids))

In [272]:
for row in range(0, len(df)):
    locations_dict = {}
    ID = df.at[row, 'ID']
    filtered_df = data.loc[data['annotation'] == 'start'].set_index('ID')
    
    for item in all_items:
        locations_dict[item] = [filtered_df.at[ID, str(item) + '_x'],
                               filtered_df.at[ID, str(item) + '_y'], 2.0]
    df.at[row, 'coordinates'] = locations_dict

In [273]:
for row in range(0, len(df)):
    subject_locations = []
    ID = df.at[row, 'ID']
    filtered_annotation = data.loc[data['ID'] == ID].reset_index()
    
    for row2 in range(0, len(filtered_annotation)):
        if filtered_annotation.at[row2, 'annotation'] == 'start':
            subject_locations.append([filtered_annotation.at[row2, 'head_x'],
                                      filtered_annotation.at[row2, 'head_y'], 2.0])
        elif filtered_annotation.at[row2, 'annotation'] == 'nehmen_objekte':
            subject_locations.append([filtered_annotation.at[row2 - 1, 'head_x'],
                                     filtered_annotation.at[row2 - 1, 'head_y'], 2.0])
    
    #print(subject_locations)
    df.at[row, 'start_coordinates'] = subject_locations

In [275]:
object_cols = ['erstes_objekt', 'zweites_objekt', 'drittes_objekt', 
               'viertes_objekt', 'fuenftes_objekt']

for row in range(0 ,len(df)):
    ID = df.at[row, 'ID']
    filtered_annotation = data.loc[(data['ID'] == ID) & (data['annotation'] == 'nehmen_objekte')].reset_index()
    sequence = []
    
    for row2 in range(0, len(filtered_annotation)):
        if pd.isna(filtered_annotation.at[row2, 'zweites_objekt']):
            sequence.append(filtered_annotation.at[row2, 'erstes_objekt'])
        else:
            set_name = str('set') + '_' + str(row2)
            sequence.append(set_name)
            set_items = []
            
            if set_name not in df.columns:
                df[set_name] = None
            
            for col in object_cols:
                if not pd.isna(filtered_annotation.at[row2, col]):
                    set_items.append(filtered_annotation.at[row2, col])
                    
            df.at[row, set_name] = set_items
        
    df.at[row, 'sequence'] = sequence 

In [277]:
df.to_csv('kit_task_envs_2022-08-26.csv', header=True, index=False)